In [4]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import gurobipy as gp

In [5]:
data = pd.read_csv('./Data/Simulated_Data_WithNoise.csv').iloc[501,:]

In [6]:
data

T    -762.627583
A    -770.292401
B       7.664819
AA   -721.319846
AB    -48.972555
BA     -3.088646
BB     10.753465
Name: 501, dtype: float64

In [7]:
y1 = np.array(data.tolist())
y1

array([-762.62758284, -770.2924015 ,    7.66481866, -721.31984618,
        -48.97255531,   -3.08864598,   10.75346464])

In [8]:
y = np.matrix([y1]*1000).T
y.shape

(7, 1000)

In [9]:
import json
with open('./Base_Forecasts/WithNoise_ARIMA.json') as file:
    fc = json.load(file)

In [10]:
len(fc)

10

In [11]:
mean = fc[0][0]

In [12]:
var = fc[0][1]
var

[33.38633277923239,
 99.79343364803083,
 57.28274672135659,
 93.40534218249954,
 85.54364750003883,
 51.103992783204,
 48.64458417576695]

In [13]:
cov = np.diag(var)
cov

array([[33.38633278,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 99.79343365,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 57.28274672,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 93.40534218,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 85.5436475 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        51.10399278,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , 48.64458418]])

In [14]:
from numpy import random
x = random.multivariate_normal(mean, cov, 1000).T
xs = random.multivariate_normal(mean, cov, 1000).T
x.shape

(7, 1000)

In [15]:
print(x[0][0])
print(xs[1][0])

-766.4057777601772
-776.8513373678909


In [16]:
S = np.matrix(np.array([[0,-1,-1,1],[0,1,1,0],[-1,-1,-1,1],[1,0,0,0],
                        [0,1,0,0],[0,0,1,0],[0,0,0,1]]))
S

matrix([[ 0, -1, -1,  1],
        [ 0,  1,  1,  0],
        [-1, -1, -1,  1],
        [ 1,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]])

In [17]:
new_index = [1,2,3,4,5,6,0]

In [18]:
x1 = np.take(x, new_index, axis=0)
x2 = np.take(xs, new_index, axis=0)
y = np.take(y, new_index, axis=0)
print(x1[6][0])

-766.4057777601772


In [19]:
""" SUPPRESS ALL OUTPUT """
OutputFlag=0
env = gp.Env(empty=True)
env.setParam("OutputFlag",OutputFlag)
env.start()

""" GLASSO MODEL """
model = gp.Model('GLASSO', env=env) # the optimization model

In [20]:
from gurobipyv1 import GRB, quicksum
p = 3  # 行数
q = 7  # 列数

G = []

for i in range(p):
    row = []
    for j in range(q):
        # 创建一个新的变量，vtype 设置为连续型，上界为1，下界为-1
        var = model.addVar(ub=1, lb=-1, vtype=GRB.CONTINUOUS, name=f"G_({i}, {j})")
        row.append(var)
    G.append(row)
G1 = np.matrix(G)

In [21]:
G1.shape
G1 = np.append(G1,[[0,0,0,0,0,0,1]],axis=0)
G1

matrix([[<gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>],
        [<gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>],
        [<gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*>],
        [0

In [22]:
m1 = np.linalg.norm(x1-x2)**2
np.sum(np.square(x1-x2))

949455.3725354916

In [23]:
y.shape

(7, 1000)

In [24]:
def squa(y):
    s=0
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            s+=y[i,j]*y[i,j]
    return s

In [25]:
part1 = squa(0.5 * (S @ G1 @ x1 - S @ G1 @ x2))
part2 = squa((y - S @ G1 @ x1))

model.setObjective((part1 + part2)/1000, GRB.MINIMIZE)

In [26]:
model.optimize()

In [27]:
model.objVal

203.6230945446723

In [28]:
variables = model.getVars()
print(len(variables))

for var in variables:
    print(var)

21
<gurobi.Var G_(0, 0) (value -0.03377553604443484)>
<gurobi.Var G_(0, 1) (value 0.002460735021447835)>
<gurobi.Var G_(0, 2) (value -0.03438089750515405)>
<gurobi.Var G_(0, 3) (value -0.003522116487747029)>
<gurobi.Var G_(0, 4) (value 0.000901250971862444)>
<gurobi.Var G_(0, 5) (value -0.00792592814024684)>
<gurobi.Var G_(0, 6) (value 0.13254427969909854)>
<gurobi.Var G_(1, 0) (value -0.044713608443609276)>
<gurobi.Var G_(1, 1) (value 0.003257634233597706)>
<gurobi.Var G_(1, 2) (value -0.04551501430415883)>
<gurobi.Var G_(1, 3) (value -0.004662739881785671)>
<gurobi.Var G_(1, 4) (value 0.0011931174953796475)>
<gurobi.Var G_(1, 5) (value -0.01049270841456651)>
<gurobi.Var G_(1, 6) (value 0.09450671212167316)>
<gurobi.Var G_(2, 0) (value -0.04801337184241994)>
<gurobi.Var G_(2, 1) (value 0.003498040281034509)>
<gurobi.Var G_(2, 2) (value -0.04887391886006698)>
<gurobi.Var G_(2, 3) (value -0.005006839496685345)>
<gurobi.Var G_(2, 4) (value 0.0012811668818315525)>
<gurobi.Var G_(2, 5) (va